In [1]:
import time
import pickle
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import folium
import pytz
import datetime as dt
import ast
import plotly.graph_objects as go
import os
import random

from stravalib import Client
pd.set_option("display.max_columns", 100)

In [2]:
# Get Data
with open('~/strava-spotify-tracking/access_tokens/config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

# Access the configuration data
client_id = config['STRAVA_API']['client_id']
client_secret = config['STRAVA_API']['client_secret']
redirect_uri = config['STRAVA_API']['redirect_uri']

client = Client()
authorize_url = client.authorization_url(
    client_id=client_id, redirect_uri=redirect_uri,
    scope=['read_all','profile:read_all','activity:read_all']
)

print(authorize_url)

https://www.strava.com/oauth/authorize?client_id=105395&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauthorization&approval_prompt=auto&scope=read_all%2Cprofile%3Aread_all%2Cactivity%3Aread_all&response_type=code


In [5]:
# authorize_url
CODE = '4210381afd4520000fdb50a86ab772595b5d733e'

In [6]:
access_token = client.exchange_code_for_token(client_id=client_id, client_secret=client_secret, code=CODE)
with open('../access_token.pickle', 'wb') as f:
    pickle.dump(access_token, f)
    

In [7]:
with open('../access_token.pickle', 'rb') as f:
    access_token = pickle.load(f)
    
print('Latest access token read from file:')
access_token


if time.time() > access_token['expires_at']:
    print('Token has expired, will refresh')
    refresh_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID, 
                                               client_secret=MY_STRAVA_CLIENT_SECRET, 
                                               refresh_token=access_token['refresh_token'])
    access_token = refresh_response
    with open('../access_token.pickle', 'wb') as f:
        pickle.dump(refresh_response, f)
    print('Refreshed token saved to file')

    client.access_token = refresh_response['access_token']
    client.refresh_token = refresh_response['refresh_token']
    client.token_expires_at = refresh_response['expires_at']
        
else:
    print('Token still valid, expires at {}'
          .format(time.strftime("%a, %d %b %Y %H:%M:%S %Z", time.localtime(access_token['expires_at']))))

    client.access_token = access_token['access_token']
    client.refresh_token = access_token['refresh_token']
    client.token_expires_at = access_token['expires_at']

Latest access token read from file:
Token still valid, expires at Sat, 24 Feb 2024 20:35:42 EST


In [ ]:
athlete = client.get_athlete()
print("Athlete's name is {} {}, based in {}, {}"
      .format(athlete.firstname, athlete.lastname, athlete.city, athlete.country))

activities = client.get_activities(limit=1000)
list(activities)[0:10]

In [14]:
# Choose some fields of interest from this data in order to read into a DataFrame
my_cols =['name',
          'start_date_local',
          'type',
          'distance',
          'moving_time',
          'elapsed_time',
          'total_elevation_gain',
          'elev_high',
          'elev_low',
          'average_speed',
          'max_speed',
          'average_heartrate',
          'max_heartrate',
          'start_latlng']

data = []
for activity in activities:
    my_dict = activity.to_dict()
    data.append([activity.id]+[my_dict.get(x) for x in my_cols])
    
# Add id to the beginning of the columns, used when selecting a specific activity
my_cols.insert(0,'id')

In [15]:
def get_hr_stream(activities_df, stream_types: list):
    ''' GET STREAM DATA FOR HEARTRATE ON STRAVA ACTIVITIES'''
    return_stream = {}

    for id in activities_df.id:
        try:
            streams = client.get_activity_streams(id, types=stream_types, resolution="medium")
            if streams:
                if "time" in streams.keys() and "heartrate" in streams.keys():
                    return_stream[id] = {
                        'time': list(streams["time"].data),
                        'heartrate': list(streams["heartrate"].data)
                    }
        except:
            pass

    return return_stream



In [16]:

# Load Existing Activity Data from CSV
cwd = os.getcwd()

activities_df = pd.read_csv(f'{cwd}/user_data/strava_activities.csv')
df_activity = activities_df[['id','name','distance', 'start_date','start_date_local','elapsed_time','end_date']]

stream_types = ["time", "heartrate"]
d = get_hr_stream(df_activity,stream_types)
df_heartrate = pd.DataFrame.from_dict(d, orient='index')

existing_csv_path = f"{cwd}/user_data/strava_heartrate.csv"
if os.path.exists(existing_csv_path):
    existing_df = pd.read_csv(existing_csv_path)
    existing_ids = existing_df['id'].tolist()
    new_ids = [id for id in df_heartrate.index if id not in existing_ids]
    new_df = df_heartrate.loc[new_ids]
    new_df.to_csv(existing_csv_path, mode='a', header=False, index=False)
else:
    df_heartrate.to_csv(existing_csv_path, index=False)